In [86]:
import gym
from gym import spaces
import numpy as np
from nasbench import api

2022-07-22 08:25:21.298020: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-22 08:25:21.298092: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [87]:
dataset = api.NASBench("/scratch2/sem22hs2/nasbench_full.tfrecord")

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


KeyboardInterrupt: 

In [5]:
def objective_function(adjacency_mat,labeling, budget=108):
    labeling = ['input'] + list(labeling) + ['output']
    model_spec = api.ModelSpec(adjacency_mat, labeling)
    try:
        data = dataset.query(model_spec, epochs=budget)
    except api.OutOfDomainError:
        # self.record_invalid(adjacency_mat, labeling, 1, 1, 0)
        return 0, 0

    # self.record_valid(adjacency_mat, labeling, data, model_spec)
    return data["validation_accuracy"], data["training_time"]

In [6]:
# Todo: sample architecture
# Todo: Check architecture validity. Neg reward if not.

class NasBench101(gym.Env):
    metadata = {"render_modes": [], "render_fps": 1}
    def __init__(self, v=7, e=9, ops=['conv1x1-bn-relu', 'conv3x3-bn-relu', 'maxpool3x3'], step_max=1000, 
                 render_mode: Optional[str] = None):
        assert render_mode is None # or render_mode in self.metadata["render_modes"]
        # Environment definition
        self.max_edges = e
        self.vertices = v
        self.ops = ops

        # Current state
        self.adjacency_mat = np.zeros([v,v])
        self.labeling = (v-2)*[ops[0]] # Initialize op for all layers that are not input or output layer
        
        # Helper
        self.idx_upper = np.triu_indices(v) # Indices of upper triangular matrix

        self.num_step = 0
        self.step_max =  step_max
        num_indecies_triu = len(self.idx_upper[0])
        self.observation_space = spaces.Dict(
            {
                "adjacency_mat": spaces.MultiBinary(num_indecies_triu),
                "labels": spaces.MultiDiscrete(np.array((v-2)*[len(ops)])),
            }
        )

        
    def step(self, action):
        e=self.max_edges
        v=self.vertices
        n = (v*(v+1)/2) # Number of indices in upper triag. part of matrix
        if action < n:
            # Todo: Check this changes matrix at right place
            iu = self.idx_upper
            self.adjacency_mat[iu[0][action],iu[1][action]] = not self.adjacency_mat[iu[0][action],iu[1][action]]
        else:
            o=len(self.ops)
            action = action - n
            [label_row, op] = np.unravel_index(action,[v, o])
            self.labeling[label_row] = op

        y, c = objective_function(self.adjacency_mat, self.labeling)
        reward = y
        if self.step == self.step_max:
            done = 1
        else: done = 0

        observation = None
        info = None
        return observation, reward, done, info

    def reset(self):
        self.adjacency_mat[self.idx_upper] = np.random.randint(0,2,len(self.idx_upper))
        self.labeling = np.random.randint(0,3,len(self.labeling))

In [135]:
# Create the API instance for the topology search space in NATS
from nats_bench import create
api = create("/scratch2/sem22hs2/NATS-tss-v1_0-3ffb9-simple", 'tss', fast_mode=True, verbose=False)

In [ ]:
# Show the architecture topology string of the 12-th architecture
# For the topology search space, the string is interpreted as
# arch = '|{}~0|+|{}~0|{}~1|+|{}~0|{}~1|{}~2|'.format(
#         edge_node_0_to_node_1,
#         edge_node_0_to_node_2,
#         edge_node_1_to_node_2,
#         edge_node_0_to_node_3,
#         edge_node_1_to_node_3,
#         edge_node_2_to_node_3,
#         )

In [136]:
architecture_str = api.arch(12)
print(architecture_str)

|none~0|+|none~0|none~1|+|none~0|nor_conv_3x3~1|avg_pool_3x3~2|


In [137]:
info = api.get_more_info(architecture_str, 'cifar10')
print(info)


{'train-loss': 2.302620864105225, 'train-accuracy': 9.866, 'train-per-time': 13.317416548728943, 'train-all-time': 159.80899858474731, 'comment': 'In this dict, train-loss/accuracy/time is the metric on the train+valid sets of CIFAR-10. The test-loss/accuracy/time is the performance of the CIFAR-10 test set after training on the train+valid sets by 12 epochs. The per-time and total-time indicate the per epoch and total time costs, respectively.', 'test-loss': 2.3025942649841307, 'test-accuracy': 10.0, 'test-per-time': 1.0880018813269479, 'test-all-time': 13.056022575923373}


In [130]:
from typing import Optional
class NasBench201Tensor(gym.Env):
    metadata = {"render_modes": [], "render_fps": 1}
    def __init__(self, step_max=1000, render_mode: Optional[str] = None):
        assert render_mode is None # or render_mode in self.metadata["render_modes"]
        
        # Environment definition
        self.vertices = 4
        self.ops = ['nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3', 'skip_connect']

        # Current state
        v = self.vertices
        no_ops = len(self.ops)
        self.adjacency_tensor = np.zeros([v,v,no_ops])
        
        # Helper
        self.triu_y, self.triu_x = np.triu_indices(v, 1) # Indices of upper triangular matrix
        self.num_triu = len(self.triu_y) # Number of upper triangular elements in matrix
        num_triu = self.num_triu
        self._label_to_op = {
            0: 'nor_conv_1x1',
            1: 'nor_conv_3x3',
            2: 'avg_pool_3x3',
            3: 'skip_connect',
        }
        self.observation_space = spaces.MultiBinary(no_ops * num_triu)
        self.action_space = spaces.Discrete(no_ops * num_triu)
        
        # Len intervals
        self.num_step = 0
        self.step_max =  step_max
        
        #if self.render_mode == "human":
        #    pass
        return
        
    def _tensor2obs(self, tensor):
        obs=[]
        for i in range(tensor.shape[2]):
            obs.extend(tensor[self.triu_x,self.triu_y,i])
        return obs
    
    def _action2tensor(self, action):
        no_triu = self.num_triu
        tensor_z = int(action/no_triu)
        triu_idx = action%no_triu
        tensor_x = self.triu_x[triu_idx]
        tensor_y = self.triu_y[triu_idx]
        
        element =  self.observation_space[action]
        new_element = not element
        if new_element is True:
            # Enforce whole row to be zero
            self.adjacency_tensor[tensor_y, tensor_x, :] = np.zeros(4, dtype=int)
        self.adjacency_tensor[tensor_y, tensor_x, tensor_z] = new_element
        return
    
    def ten2str(self,tensor):
        adj_str = "|"
        for j in range(tensor.shape[1]): #to
            for k in range(tensor.shape[0]): #from
                if j <= k:
                    continue
                if k==0 and j != 1:
                    adj_str += "+|"
                    
                for i in range(tensor.shape[2]):#op
                    if not tensor[:,k,j].any():
                        adj_str += "none~"+str(k)+"|"
                        break
                    if tensor[i,k,j] == 1:
                        adj_str +=_label_to_op[i] +"~"+str(k)+ "|"
        return adj_str
    
    def nb201_lookup(self,tensor):
        arch=self.ten2str(tensor)
        acc=api.get_more_info(architecture_str, 'cifar10')
        return acc
                           
                       
    def step(self, action):
        # Determine new adjacency matrix and observation space
        self._action2tensor(action)
        self.observation = self._tensor2obs(self.adjacency_tensor)
        # Calculate reward
        acc = self.nb201_lookup(self.adjacency_tensor)
        reward = acc
        if self.step == self.step_max:
            done = 1
        else: done = 0

        observation = self.observation
        info = None
        return observation, reward, done, info

    def reset(self):
        v = self.vertices
        no_ops = len(self.ops)
        self.adjacency_tensor = np.zeros([v,v,no_ops])
        self.adjacency_tensor[0,3] = 1
        self.obervation=self._tensor2obs(self.adjacency_tensor)
        self.num_step = 0
        return self.obervation
        

In [102]:
import ray
from ray.tune.integration.wandb import WandbLoggerCallback

In [92]:
SELECT_ENV = NasBench201Tensor
N_ITER = 20


ray.init()
ray.tune.run(
    "PPO",
    stop={"training_iteration": 15},
    config={
        "env": SELECT_ENV,
        "record_env": True,
        "framework": "torch",
        "num_cpus_per_worker": 2,
        "num_gpus": 2,
        "num_workers": 4,
    },
    #local_dir="logs",
    #callbacks=[WandbLoggerCallback(api_key="c36c598399c6c7f2f0b446aac164da6c7956a263", project="NasBenchV1")],
)


trainer.evaluate()

/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
(PPOTrainer pid=14980) 2022-07-22 08:33:30,537	INFO ppo.py:414 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=14980) 2022-07-22 08:33:30,538	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=15028) 2022-07-22 08:33:36,116	ERROR worker.py:451 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=15028, ip=129.132.4.157, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7feaccf802e0>)
(RolloutWorker pi

Trial name,status,loc
PPO_NasBench201Tensor_301de_00000,RUNNING,


2022-07-22 08:33:36,385	ERROR trial_runner.py:886 -- Trial PPO_NasBench201Tensor_301de_00000: Error processing event.
NoneType: None


Result for PPO_NasBench201Tensor_301de_00000:
  trial_id: 301de_00000
  


(PPOTrainer pid=14980) 2022-07-22 08:33:36,350	ERROR worker.py:451 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPOTrainer.__init__() (pid=14980, ip=129.132.4.157, repr=PPOTrainer)
(PPOTrainer pid=14980)   File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 1074, in _init
(PPOTrainer pid=14980)     raise NotImplementedError
(PPOTrainer pid=14980) NotImplementedError
(PPOTrainer pid=14980) 
(PPOTrainer pid=14980) During handling of the above exception, another exception occurred:
(PPOTrainer pid=14980) 
(PPOTrainer pid=14980) ray::PPOTrainer.__init__() (pid=14980, ip=129.132.4.157, repr=PPOTrainer)
(PPOTrainer pid=14980)   File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 870, in __init__
(PPOTrainer pid=14980)     super().__init__(
(PPOTrainer pid=14980)   File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/tune/train

Trial name,status,loc
PPO_NasBench201Tensor_301de_00000,ERROR,
Trial name,# failures,error file
PPO_NasBench201Tensor_301de_00000,1,/home/sem22h2/ray_results/PPO/PPO_NasBench201Tensor_301de_00000_0_2022-07-22_08-33-24/error.txt


Trial name,status,loc
PPO_NasBench201Tensor_301de_00000,ERROR,
Trial name,# failures,error file
PPO_NasBench201Tensor_301de_00000,1,/home/sem22h2/ray_results/PPO/PPO_NasBench201Tensor_301de_00000_0_2022-07-22_08-33-24/error.txt


2022-07-22 08:33:36,460	ERROR ray_trial_executor.py:107 -- An exception occurred when trying to stop the Ray actor:Traceback (most recent call last):
  File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/tune/ray_trial_executor.py", line 98, in post_stop_cleanup
    ray.get(future, timeout=0)
  File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/worker.py", line 1833, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::PPOTrainer.__init__() (pid=14980, ip=129.132.4.157, repr=PPOTrainer)
  File "/home/sem22h2/.conda/envs/RL/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 1074, in _init
    raise NotImplementedError
NotImplementedError

During handling of the above exception, another exception occurred:

ray::PPOTrainer.__i

TuneError: ('Trials did not complete', [PPO_NasBench201Tensor_301de_00000])

In [ ]:
ray.shutdown()

In [12]:
b = NasBench(v=3)

In [14]:
b.step(1)

(None, 0, 0, None)

In [16]:
b=[1,2,3]

In [16]:
import numpy as np
y,x =np.triu_indices(4,1)
mat=np.arange(16).reshape(4, 4)

In [20]:
mat[y[0],x[0]]=90

In [117]:
from gym import spaces
b=spaces.MultiBinary(3 * 3)
b.sample()[0]

0

In [115]:
b[0]

TypeError: 'MultiBinary' object is not subscriptable

In [49]:
print(p[:,1,2])
print(p)

[0 1 0]
[[[0 1 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 1]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]]]


In [131]:
a=NasBench201Tensor()
a.reset()
a.step(3)

TypeError: 'MultiBinary' object is not subscriptable